In [2]:
import numpy as np
from scipy.spatial.distance import cdist
import time
from sklearn.preprocessing import LabelBinarizer
import random as ran
import time
import torch

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [3]:
class RBF(object):

    def __init__(self, lengthscale=1., variance=1., noise=1.):
        super(RBF, self).__init__()
        self.lengthscale = lengthscale
        self.variance = variance
        self.jitter = 1e-9
        # self.noise = noise / self.variance + self.jitter# dividing by variance for new strategy
        self.noise = noise

    def K(self, X1, X2):
        """ GP squared exponential kernel """
        pairwise_dists = cdist(X1, X2, 'sqeuclidean')
        print(type(pairwise_dists[0,0]))
        return self.variance*np.exp(-0.5 * pairwise_dists / self.lengthscale ** 2)
        # return pairwise_dists

In [6]:
torch.cuda.device_count()

2

In [4]:
"""
Solve linear system using conjugate gradient
Params:
    K - Covariance Matrix
    Y - Target labels
    init - Initial solution
    thershold - Termintion criteria
"""
def MultiCGGPU(K, Y, init=None, tol=1e-5, atol=1e-9, max_iterations=15000, cuda=False, num_gpus=3):
    N = np.shape(K)[0]
    if init is None:
        init = np.zeros((N,1))

    # self.K = K
    # self.Y = Y.flatten()


    x = init
    R = Y - np.dot(K, x) #initialise residual gradient

    # Move two kernel splits to 2 GPUs
    K = torch.from_numpy(K).type(torch.FloatTensor) #.cuda()
    split_size = K.shape[0] // num_gpus
    if cuda:
        Ks = []
        for i in range(num_gpus):
            if i < (num_gpus-1):
                Ks.append(
                    K[i*split_size:(i+1)*split_size].to('cuda:' + str(i))
                )
            else:
                Ks.append(
                    K[i*split_size:].to('cuda:' + str(i))
                )

    iterations = []
    solutions = []

    for dim in range(Y.shape[1]):
        print('Starting CG for dimension {}'.format(dim))
        since = time.time()
        # get current residual vector
        r = R[:, dim][:, None]

        p = r

        t = 0

        x = torch.from_numpy(init).type(torch.FloatTensor) #.cuda()
        r = torch.from_numpy(r).type(torch.FloatTensor) #.cuda()
        p = torch.from_numpy(p).type(torch.FloatTensor) #.cuda()

        if cuda:
            x = x.to('cuda:0')
            r = r.to('cuda:0')
            ps = [p.to('cuda:' + str(i)) for i in range(num_gpus)]

        while True:
            with torch.no_grad():
                # alpha = np.dot(r.T, r) / np.dot(p.T, np.dot(K, p))
                Kps = [Ks[i].mm(ps[i]).to('cuda:0') for i in range(num_gpus)]
                Kp = torch.cat(Kps, dim=0)
                pKp = ps[0].t().mm(Kp)

                alpha = r.t().mm(r) / pKp
                x = x + alpha*ps[0]
                r_prev = r
                # r = r - alpha*np.dot(K, p)
                r = r - alpha * Kp

                # if ((np.dot(r.T,r).flatten() < (threshold*N)) or (t>15000)):
                if ((r.t().mm(r).item() <= max(tol*np.linalg.norm(Y[:, dim]), atol*N)) or (t>max_iterations)):
                    break
                # if ((r.t().mm(r).item() < (threshold*N)) or (t>max_iterations)):
                #     break
                # beta = np.dot(r.T, r) / np.dot(r_prev.T, r_prev)
                beta = r.t().mm(r) / r_prev.t().mm(r_prev)
                ps[0] = r + beta*ps[0]

                # we need to send the updated p to gpu_i (two vector transfers in total)
                ps = [ps[0].to('cuda:' + str(i)) for i in range(num_gpus)]

                t = t + 1

        print('Iterations needed: {}'.format(t))
        print('Time elapsed: {}'.format(time.time() - since))
        iterations.append(t)
        if cuda:
            x = x.cpu()
        solutions.append(x.numpy())

    return np.hstack(solutions), iterations

In [5]:
# (train_data, train_labels), (test_data, test_labels) = FashionMNIST()

train_data = np.load('../../datasets/export/fashion_mnist/numpy/train_data_fashion_mnist.npy').astype('float32')
test_data = np.load('../../datasets/export/fashion_mnist/numpy/test_data_fashion_mnist.npy').astype('float32')
train_labels = np.load('../../datasets/export/fashion_mnist/numpy/train_targets_fashion_mnist.npy').astype('float32')
test_labels = np.load('../../datasets/export/fashion_mnist/numpy/test_targets_fashion_mnist.npy').astype('float32')

train_data = train_data[:60000]
train_labels = train_labels[:60000]

# Convert one-hot to integers
train_labels = np.argmax(train_labels, axis=1)
test_labels = np.argmax(test_labels, axis=1)

D = train_data[0].reshape(-1).shape[0]
N = len(train_data)

# Flatten the images
train_data = train_data.reshape(-1, D)
test_data = test_data.reshape(-1, D)

In [6]:
def threshold_binarize(data, threshold):
    data_bin = np.where(data>threshold, 1, 0)
    return data_bin

threshold = 10

train_data_bin = threshold_binarize(train_data, threshold).astype('float32')
test_data_bin = threshold_binarize(test_data, threshold).astype('float32')

In [1]:
active_pixels_train = train_data_bin.sum(axis=1, keepdims=True)
active_pixels_test = test_data_bin.sum(axis=1, keepdims=True)

NameError: name 'train_data_bin' is not defined

In [7]:
label_binarizer = LabelBinarizer(pos_label=1, neg_label=-1)
train_labels_bin = label_binarizer.fit_transform(train_labels).astype('float32')
test_labels_bin = label_binarizer.fit_transform(test_labels).astype('float32')

In [24]:
# M = int(np.sqrt(N))
# ipHelper = InducingPointsHelper(0)
# XmRandom = ipHelper.get_random_inducing_points(train_data_bin,M)

In [13]:
# XmRandom.shape

(100, 784)

In [50]:
# kernel = RBF(lengthscale=np.sqrt(D/2), variance=1., noise=1e-6)

In [26]:
# jitter = 1e-6
# K = kernel.K(train_data_bin, train_data_bin) + jitter*np.identity(N)

In [51]:
# 5K estimates
kernel_var = 0.1789339259732666
lengthscale = 7.177267570983543
jitter = 0.04144659858222422
gamma = 1. / (2*lengthscale**2)

In [59]:
gamma

0.00970625574163014

In [36]:
jitter

0.04

In [8]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel

In [12]:
# after grid search

gamma = 0.006
var = 1.
alpha = 0.1

# 0.8839 score

K = var * rbf_kernel(train_data_bin, Y=train_data_bin, gamma=gamma) + alpha*np.identity(N).astype('float32')

In [9]:
def opu_kernel(x, y, gamma=1):
    kernel = polynomial_kernel(x, Y=y, degree=2, gamma=1, coef0=0)
    norm_x_sq = np.linalg.norm(x, ord=2, axis=1, keepdims=True) ** 2
    norm_y_sq = np.linalg.norm(y, ord=2, axis=1, keepdims=True) ** 2

    # corresponds to element-wise addition of norm_x^2 * norm_y^2
    kernel += np.dot(norm_x_sq, norm_y_sq.T)
    
    kernel *= gamma
    
    return kernel

In [9]:
def opu_kernel_two(x, y, gamma=1):
    poly = polynomial_kernel(x, Y=y, degree=2, gamma=1, coef0=0)
    norm_x_sq = np.linalg.norm(x, ord=2, axis=1, keepdims=True) ** 2
    norm_y_sq = np.linalg.norm(y, ord=2, axis=1, keepdims=True) ** 2
    norm_matrix_sq = np.dot(norm_x_sq, norm_y_sq.T)
    
    return gamma*(4*norm_matrix_sq**2 + 16*norm_matrix_sq*poly + 4*poly**2)

In [10]:
# we chose gamma according to the random feature optimization step
gamma = 0.001**2 * 784
# alpha is chosen like that as well
alpha = 10
K = opu_kernel(train_data_bin, y=train_data_bin, gamma=gamma) + alpha*np.identity(N).astype('float32')

In [10]:
gamma = 0.001**4 * 784
alpha = 10
K = opu_kernel_two(train_data_bin, y=train_data_bin, gamma=gamma) + alpha*np.identity(N).astype('float32')

In [11]:
K.shape

(60000, 60000)

In [25]:
# import bunch

In [26]:
# kern = bunch.Bunch()

In [27]:
# kern.K = K

In [28]:
# kern.noise = jitter
# kern.jitter = 1e-9

In [64]:
# originally adding noise
# K = kernel.K(train_data_bin,train_data_bin) + kernel.noise*np.identity(N).astype('float32')

<class 'numpy.float64'>


In [23]:
type(K[0,0])

numpy.float32

In [54]:
# kernel.noise

1e-06

In [24]:
K

array([[267.19116 , 197.69812 , 134.91069 , ..., 125.52937 , 214.44438 ,
         71.30087 ],
       [197.69812 , 337.4752  , 187.59235 , ..., 181.74216 , 321.19772 ,
         69.19427 ],
       [134.91069 , 187.59235 , 166.57419 , ..., 134.55635 , 198.69382 ,
         47.140347],
       ...,
       [125.52937 , 181.74216 , 134.55635 , ..., 159.7142  , 193.56255 ,
         45.5896  ],
       [214.44438 , 321.19772 , 198.69382 , ..., 193.56255 , 386.4768  ,
         75.47255 ],
       [ 71.30087 ,  69.19427 ,  47.140347, ...,  45.5896  ,  75.47255 ,
         54.783646]], dtype=float32)

In [25]:
type(K[0,0])

numpy.float32

In [147]:
# prec = Nystrom(train_data_bin, kern, XmRandom)

Type: <class 'numpy.float32'>
Size: (10000, 100) (100, 100)


In [102]:
# prec.precon.shape

(10000, 10000)

In [148]:
# inv = prec.get_inversion()

In [81]:
# inv.shape

(5000, 5000)

In [26]:
# train_labels[:, None].shape

(5000, 1)

In [ ]:
# 1K dimensions: 1 second/label
# 10K dimensiosn: 1724 seconds/label (0.5 hours)
# in theory: 1000x slower

In [12]:
since = time.time()
# print('Running CG for dim', dim)
# pcg = RegularPcgPyTorch(K, train_labels_bin[:, dim][:, None], prec.precon, threshold=1e-9, preconInv=inv)
# dual_coef = linalg.solve(K, train_labels_bin, sym_pos=True, overwrite_a=False)
dual_coef, iterations = MultiCGGPU(K, train_labels_bin, tol=1e-11, atol=1e-9, max_iterations=10*N, cuda=True, num_gpus=2)
# dual_cofs.append(pcg.result)
# coef, info = cg(K, train_labels_bin[:, dim], tol=1e-5) # M=inv
# coef, info = gmres(K, train_labels_bin[:, dim], tol=1e-5)
# dual_cofs.append(coef.reshape((-1, 1)))
# print('Info:', info)
print('Done. Iterations:', iterations)
print('Time:', time.time() - since)
    
# dual_coef = np.hstack(dual_cofs)

Starting CG for dimension 0
Iterations needed: 1525
Time elapsed: 26.142972946166992
Starting CG for dimension 1
Iterations needed: 1392
Time elapsed: 23.122112035751343
Starting CG for dimension 2
Iterations needed: 1546
Time elapsed: 25.789490938186646
Starting CG for dimension 3
Iterations needed: 1535
Time elapsed: 25.419824600219727
Starting CG for dimension 4
Iterations needed: 1528
Time elapsed: 25.4859881401062
Starting CG for dimension 5
Iterations needed: 1447
Time elapsed: 23.974270343780518
Starting CG for dimension 6
Iterations needed: 1565
Time elapsed: 25.968250036239624
Starting CG for dimension 7
Iterations needed: 1503
Time elapsed: 24.920526266098022
Starting CG for dimension 8
Iterations needed: 1453
Time elapsed: 24.117175579071045
Starting CG for dimension 9
Iterations needed: 1434
Time elapsed: 23.824496746063232
Done. Iterations: [1525, 1392, 1546, 1535, 1528, 1447, 1565, 1503, 1453, 1434]
Time: 366.59290409088135


In [14]:
K_test = opu_kernel_two(test_data_bin, y=train_data_bin, gamma=gamma)
prediction = np.dot(K_test, dual_coef)

In [14]:
K_test = var * rbf_kernel(test_data_bin, train_data_bin, gamma=gamma)
prediction = np.dot(K_test, dual_coef)

In [106]:
type(prediction[0,0])

numpy.float32

In [107]:
prediction.shape

(10000, 10)

In [15]:
score = np.sum(np.equal(np.argmax(prediction, 1), np.argmax(test_labels_bin, 1))) / len(test_data) * 100

In [16]:
# OPU^2: 60K!
score

89.53

In [27]:
# OPU after gpflow mini-batch optimization
# might be overrated because verification was done with test set

# m.kern.variance = 0.0018913713117732164
# m.likelihood.variance = 5.61412412417899
score

89.02

In [43]:
# 10K
score

86.21

In [25]:
# OPU: 60K!!!
score

88.55

In [37]:
score

88.51

In [27]:
K_test = variance * rbf_kernel(test_data_bin, Y=train_data_bin, gamma=gamma)
prediction = np.dot(K_test, dual_coef)

In [29]:
# rbf sparse parameters (60K inversion successful)
score

87.66000000000001

In [25]:
# rbf after proper grid search
score

89.73

## CIFAR10

In [4]:
vgg_features = np.load('conv_features/cifar10/models/vgg16_bn_avgpool.npz')

In [5]:
vgg_features['test'].shape

(10000, 25088)

In [6]:
# RBF: gamma = 0.00001, alpha = 0.10, scale = 1.0, threshold = 0.4
# OPU: alpha = 100.00, degree = 1.0, dummy = 100, scale = 0.00001, threshold = 0.4

In [5]:
def threshold_binarize(data, threshold):
    data_bin = np.where(data>threshold, 1, 0)
    return data_bin

In [6]:
threshold = 0.4

features_train = threshold_binarize(vgg_features['train'], threshold)
features_test = threshold_binarize(vgg_features['test'], threshold)

In [7]:
labels = np.load('conv_features/cifar10/labels.npz')
train_labels = labels['train']
test_labels = labels['test']

In [8]:
label_binarizer = LabelBinarizer(pos_label=1, neg_label=-1)
train_labels_bin = label_binarizer.fit_transform(train_labels).astype('float32')
test_labels_bin = label_binarizer.fit_transform(test_labels).astype('float32')

## OPU Kernel

In [22]:
import math

def compute_opu_kernel_gpu(data, data_y=None, gamma=1., dtype=torch.FloatTensor, cuda=True):
    
#     N = len(data)
#     n_chunks = math.ceil(N / chunk_size)
    
#     output_chunks = []
    
#     for i in range(n_chunks):
#         data_chunk = data[i*chunk_size:(i+1)*chunk_size]
        
#         data_chunk = torch.from_numpy(data_chunk).type(dtype)
#         if cuda:
#             data_chunk = data_chunk.cuda()

    with torch.no_grad():
        data = torch.from_numpy(data).type(dtype)
        
        if cuda:
            data = data.cuda()
            
        norm = data.norm(dim=1, keepdim=True)**2
            
        if data_y is not None:
            data_y = torch.from_numpy(data_y).type(dtype)
            if cuda:
                data_y = data_y.cuda()
                
            norm_y = data_y.norm(dim=1, keepdim=True)**2
        else:
            data_y = data
            norm_y = norm

        # do the matrix multiply in two passes to save memory
        kernel_1 = data[:data.shape[0]//2].mm(data_y.t())**2
        kernel_1 = kernel_1.cpu().numpy()
        
        kernel_2 = data[data.shape[0]//2:].mm(data_y.t())**2
        kernel_2 = kernel_2.cpu().numpy()
        
        kernel = np.vstack([kernel_1, kernel_2])
        
        # do norm matrix in two passes like above
        norm_exp_1 = norm[:norm.shape[0]//2].mm(norm_y.t())
        norm_exp_1 = norm_exp_1.cpu().numpy()
        
        norm_exp_2 = norm[norm.shape[0]//2:].mm(norm_y.t())
        norm_exp_2 = norm_exp_2.cpu().numpy()
        
        norm_exp = np.vstack([norm_exp_1, norm_exp_2])
        
        kernel += norm_exp

        return (kernel * gamma)
        
    
    
#     K = torch.from_numpy(K).type(torch.FloatTensor) #.cuda()
#     split_size = K.shape[0] // num_gpus
#     if cuda:
#         Ks = []
#         for i in range(num_gpus):
#             if i < (num_gpus-1):
#                 Ks.append(
#                     K[i*split_size:(i+1)*split_size].to('cuda:' + str(i))
#                 )
#             else:
#                 Ks.append(
#                     K[i*split_size:].to('cuda:' + str(i))
#                 )

In [39]:
import torch.nn.functional as F

def rbf_gpu(data, data_y=None, gamma=1., var=1., dtype=torch.FloatTensor, cuda=True):
    data = torch.from_numpy(data).type(dtype)

    if cuda:
        data = data.cuda()
    
    if data_y is not None:
        data_y = torch.from_numpy(data_y).type(dtype)
        if cuda:
            data_y = data_y.cuda()
    else:
        data_y = data
    
    with torch.no_grad():
#         sq_euclidean_1 = F.pairwise_distance(data[:data.shape[0]//2], data_y, 2, 1e-6, True)**2
#         sq_euclidean_1 = sq_euclidean_1.cpu().numpy()
        
#         sq_euclidean_2 = F.pairwise_distance(data[data.shape[0]//2:], data_y, 2, 1e-6, True)**2
#         sq_euclidean_2 = sq_euclidean_2.cpu().numpy()
        
#         pairwise_dists = np.vstack([sq_euclidean_1, sq_euclidean_2])

        pairwise_dists = F.pairwise_distance(data, data_y, 2, 1e-6, True)**2
        pairwise_dists = pairwise_dists.cpu().numpy()
        
        return var*np.exp(-gamma * pairwise_dists)

In [10]:
# add dummy
dummy = 100
data = np.vstack([features_train, features_test])
data = np.hstack([data, np.ones((data.shape[0], 1)) * dummy])

In [12]:
data.shape

(60000, 25089)

In [12]:
N = len(features_train)

In [23]:
# OPU:
# we chose gamma according to the random feature optimization step
gamma = 0.00001**2 * data.shape[1] # *d simulates divide by raw_scale (1/d)
# alpha is chosen like that as well
alpha = 100
# alpha = 10
K = compute_opu_kernel_gpu(data[:N], gamma=gamma) + alpha*np.identity(N).astype('float32')

In [29]:
# RBF:
gamma = 0.00001
alpha = 0.10
var = 1.0**2

K = var * rbf_kernel(data[:N], Y=data[:N], gamma=gamma) + alpha*np.identity(N).astype('float32')

In [13]:
K.shape

(50000, 50000)

In [14]:
K[0]

array([377.2977 , 262.1834 , 258.50238, ..., 263.7245 , 262.08038,
       263.15872], dtype=float32)

In [30]:
since = time.time()
# print('Running CG for dim', dim)
# pcg = RegularPcgPyTorch(K, train_labels_bin[:, dim][:, None], prec.precon, threshold=1e-9, preconInv=inv)
# dual_coef = linalg.solve(K, train_labels_bin, sym_pos=True, overwrite_a=False)
dual_coef, iterations = MultiCGGPU(K, train_labels_bin, tol=1e-15, atol=1e-15, max_iterations=10*N, cuda=True, num_gpus=2)
# dual_cofs.append(pcg.result)
# coef, info = cg(K, train_labels_bin[:, dim], tol=1e-5) # M=inv
# coef, info = gmres(K, train_labels_bin[:, dim], tol=1e-5)
# dual_cofs.append(coef.reshape((-1, 1)))
# print('Info:', info)
print('Done. Iterations:', iterations)
print('Time:', time.time() - since)
    
# dual_coef = np.hstack(dual_cofs)

Starting CG for dimension 0
Iterations needed: 148
Time elapsed: 1.8161838054656982
Starting CG for dimension 1
Iterations needed: 145
Time elapsed: 1.7687151432037354
Starting CG for dimension 2
Iterations needed: 146
Time elapsed: 1.7736530303955078
Starting CG for dimension 3
Iterations needed: 142
Time elapsed: 1.7402198314666748
Starting CG for dimension 4
Iterations needed: 141
Time elapsed: 1.7209792137145996
Starting CG for dimension 5
Iterations needed: 143
Time elapsed: 1.7327156066894531
Starting CG for dimension 6
Iterations needed: 144
Time elapsed: 1.7738502025604248
Starting CG for dimension 7
Iterations needed: 144
Time elapsed: 1.775909423828125
Starting CG for dimension 8
Iterations needed: 142
Time elapsed: 1.7249560356140137
Starting CG for dimension 9
Iterations needed: 138
Time elapsed: 1.7077176570892334
Done. Iterations: [148, 145, 146, 142, 141, 143, 144, 144, 142, 138]
Time: 28.658723831176758


In [41]:
# K_test = compute_opu_kernel_gpu(data[N:], data[:N], gamma=gamma)
# K_test = rbf_gpu(data[N:], data[:N], gamma=gamma, var=var)
K_test = var*rbf_kernel(data[N:], data[:N], gamma=gamma)
prediction = np.dot(K_test, dual_coef)

In [42]:
score = np.sum(np.equal(np.argmax(prediction, 1), np.argmax(test_labels_bin, 1))) / len(features_test) * 100

In [27]:
# OPU limit for vgg
score

87.1

In [43]:
# rbf limit for vgg
score

87.02